In [15]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import matplotlib.pyplot as plt



Matplotlib is building the font cache; this may take a moment.


## Zeitserie erstellen

In [2]:
def generate_time_eries(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  # Welle 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # Welle 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # Rauschen
    return series[..., np.newaxis].astype(np.float32)

In [4]:
n_steps = 50
series = generate_time_eries(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [10]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))


Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1541 - val_loss: 0.0570
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 826us/step - loss: 0.0423 - val_loss: 0.0264
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step - loss: 0.0211 - val_loss: 0.0148
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step - loss: 0.0133 - val_loss: 0.0101
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step - loss: 0.0093 - val_loss: 0.0081
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step - loss: 0.0077 - val_loss: 0.0070
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step - loss: 0.0068 - val_loss: 0.0061
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step - loss: 0.0060 - val_loss: 0.0054
Epoch 9/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step - loss: 0.0054 - val_loss: 0.0050
Epoch 10/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 826us/step - loss: 0.0049 - val_loss: 0.0046
Epoch 11/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - loss: 0.0046 - val_loss: 0.0044
Epoch 12/20
219/219 ━━━━━━━━━━━━

### Das einfachste RNN das man bauen könnte

In [11]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

model.compile(loss="mse", optimizer="adam")
model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20


/Users/riccardo/Desktop/Github/Bachelor/Bachelor/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.2590 - val_loss: 0.1882
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1740 - val_loss: 0.1522
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1483 - val_loss: 0.1414
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1418 - val_loss: 0.1439
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1403 - val_loss: 0.1420
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1397 - val_loss: 0.1467
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1380 - val_loss: 0.1406
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1424 - val_loss: 0.1408
Epoch 9/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1413 - val_loss: 0.1407
Epoch 10/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1402 - val_loss: 0.1415
Epoch 11/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1371 - val_loss: 0.1405
Epoch 12/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.

---------------------------------------------------------------------------

# Mehrere Zeitschritte vorhersagen

In [17]:
series = generate_time_eries(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new
for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)

y_pred = X[:, n_steps:]




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
